In [1]:
pip list

Package            Version
------------------ ---------
aiohttp            3.8.3
aiosignal          1.2.0
asttokens          2.0.8
async-timeout      4.0.2
attrs              22.1.0
backcall           0.2.0
certifi            2022.9.24
charset-normalizer 2.1.1
colorama           0.4.5
datasets           2.6.1
debugpy            1.6.3
decorator          5.1.1
dill               0.3.5.1
entrypoints        0.4
executing          1.0.0
filelock           3.8.0
frozenlist         1.3.1
fsspec             2022.8.2
huggingface-hub    0.10.0
idna               3.4
ipykernel          6.15.3
ipython            8.5.0
jedi               0.18.1
joblib             1.2.0
jupyter_client     7.3.5
jupyter-core       4.11.1
matplotlib-inline  0.1.6
multidict          6.0.2
multiprocess       0.70.13
nest-asyncio       1.5.5
numpy              1.23.3
packaging          21.3
pandas             1.5.0
parso              0.8.3
pickleshare        0.7.5
Pillow             9.2.0
pip                22.2.1
prompt


[notice] A new release of pip available: 22.2.1 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import transformers, torch, datasets
print("transformers", transformers.__version__)
print("torch", torch.__version__)
print("datasets", datasets.__version__)

transformers 4.23.1
torch 1.12.1+cu116
datasets 2.6.1


In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset = load_dataset("sst2")

Dataset sst2 downloaded and prepared to /home/lantum/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 188.76it/s]


In [10]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading: 100%|██████████| 466k/466k [00:01<00:00, 239kB/s] 
 50%|█████     | 1/2 [00:00<00:00, 11.06ba/s]


In [15]:
from torch import device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [19]:

training_args = TrainingArguments(
    output_dir="~/test",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer.train()

***** Running training *****
  Num examples = 67349
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6315
  0%|          | 5/6315 [00:02<51:50,  2.03it/s]
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
  8%|▊         | 500/6315 [03:03<32:15,  3.00it/s]Saving model checkpoint to /home/lantum/test/checkpoint-500
Configuration saved in /home/lantum/test/checkpoint-500/config.json


{'loss': 0.3227, 'learning_rate': 9.154394299287412e-06, 'epoch': 0.24}


Model weights saved in /home/lantum/test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /home/lantum/test/checkpoint-500/tokenizer_config.json
Special tokens file saved in /home/lantum/test/checkpoint-500/special_tokens_map.json
 16%|█▌        | 1000/6315 [06:09<31:32,  2.81it/s] Saving model checkpoint to /home/lantum/test/checkpoint-1000
Configuration saved in /home/lantum/test/checkpoint-1000/config.json


{'loss': 0.2356, 'learning_rate': 8.362628661916074e-06, 'epoch': 0.48}


Model weights saved in /home/lantum/test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /home/lantum/test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /home/lantum/test/checkpoint-1000/special_tokens_map.json
 24%|██▍       | 1500/6315 [09:16<30:29,  2.63it/s]Saving model checkpoint to /home/lantum/test/checkpoint-1500
Configuration saved in /home/lantum/test/checkpoint-1500/config.json


...

Model weights saved in /home/lantum/test/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in /home/lantum/test/checkpoint-5500/tokenizer_config.json
Special tokens file saved in /home/lantum/test/checkpoint-5500/special_tokens_map.json
 95%|█████████▌| 6000/6315 [37:12<02:06,  2.49it/s]Saving model checkpoint to /home/lantum/test/checkpoint-6000
Configuration saved in /home/lantum/test/checkpoint-6000/config.json


{'loss': 0.1038, 'learning_rate': 4.4497228820269205e-07, 'epoch': 2.85}


Model weights saved in /home/lantum/test/checkpoint-6000/pytorch_model.bin
tokenizer config file saved in /home/lantum/test/checkpoint-6000/tokenizer_config.json
Special tokens file saved in /home/lantum/test/checkpoint-6000/special_tokens_map.json
100%|██████████| 6315/6315 [39:11<00:00,  2.95it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6315/6315 [39:11<00:00,  2.69it/s]

{'train_runtime': 2351.3609, 'train_samples_per_second': 85.928, 'train_steps_per_second': 2.686, 'train_loss': 0.16430803904529612, 'epoch': 3.0}


TrainOutput(global_step=6315, training_loss=0.16430803904529612, metrics={'train_runtime': 2351.3609, 'train_samples_per_second': 85.928, 'train_steps_per_second': 2.686, 'train_loss': 0.16430803904529612, 'epoch': 3.0})

Сохранение модели (обученной на 6315), я хз, что я сохраняю, я хз, как это загрузить в прогу

In [52]:
torch.save(model, "/home/lantum/test1/model.bin")
torch.save(model.state_dict(),"/home/lantum/test1/model.pt")

In [41]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

Для удобства использования модели в инференсе, можно переименовать параметры с помощью словарей label2id и id2label. Это позволит при выводе результатов, видеть классы.

In [56]:
from transformers import AutoConfig, AutoModelForSequenceClassification

label2id = {
    "negative": 0,
    "positive": 1,
  }
id2label = {
    0: "negative",
    1: "positive",
  }
model_ckpt = "/home/lantum/test/checkpoint-6000"
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)


loading configuration file /home/lantum/test/checkpoint-6000/config.json
Model config DistilBertConfig {
  "_name_or_path": "/home/lantum/test/checkpoint-6000",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}



Загружаем сохраненную модель (!здесь на 6000!)

In [59]:
from transformers import DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/home/lantum/test/checkpoint-6000")
model = AutoModelForSequenceClassification.from_pretrained("/home/lantum/test/checkpoint-6000", config=config)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading weights file /home/lantum/test/checkpoint-6000/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at /home/lantum/test/checkpoint-6000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


Проверяем предсказание модели на тестовом предложении test

In [61]:
text = "Artemiy Mazaev is a fucking asshole! I HATE HIM!"

tensor = tokenizer(text, padding="max_length",  truncation=True, max_length=512, return_tensors="pt")


print("Example output", model(**tensor))

logits = model(**tensor).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

Example output SequenceClassifierOutput(loss=None, logits=tensor([[ 2.4995, -2.4793]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


'negative'